In [ ]:
from pytket import Circuit
from pytket.circuit import OpType
from pytket.circuit.display import render_circuit_jupyter
from pytket_dqc.networks import NISQNetwork
from pytket_dqc.allocators import HypergraphPartitioning
from pytket_dqc.circuits import HypergraphCircuit
from pytket_dqc.packing import PacMan
from pytket_dqc.utils import DQCPass
from sympy import Symbol
from pytket.passes import SynthesiseTK

In [ ]:
import json
with open("06_02_01.json") as f:
    circuit = Circuit.from_dict(json.load(f))
network = NISQNetwork(
    [[0,1]],
    {0: [0,1,2], 1: [3,4,5]}
)
t01 = Symbol("t0_1")
t02 = Symbol("t0_2")
t11 = Symbol("t1_1")
t12 = Symbol("t1_2")
t22 = Symbol("t2_2")
t21 = Symbol("t2_1")
t31 = Symbol("t3_1")
t41 = Symbol("t4_1")
t51 = Symbol("t5_1")
t61 = Symbol("t6_1")
t71 = Symbol("t7_1")
circ_map = {t01: 0, t02:0, t11: 0, t21: 0, t12: 0, t22:0, t31: 0, t41: 0, t51: 0, t61: 0, t71: 0}
circuit.symbol_substitution(circ_map)
render_circuit_jupyter(circuit)
circuit.free_symbols()

In [ ]:
DQCPass().apply(circuit)
print(len(circuit.get_commands()))
print(circuit.n_gates_of_type(OpType.CU1))
render_circuit_jupyter(circuit)
hyper_circ = HypergraphCircuit(circuit)
distributor = HypergraphPartitioning()
placement = distributor.allocate(hyper_circ, network)
print(placement.placement.placement)

In [ ]:
pacman = PacMan(hyper_circ, placement.placement)

In [ ]:
print(pacman.packets_by_qubit)
print(pacman.hopping_packets)
total_sum = 0

for value in pacman.hopping_packets.values():
    total_sum += len(value)
print(total_sum)

In [ ]:
import networkx as nx
from networkx.algorithms import bipartite
g, topnodes = pacman.get_nx_graph_neighbouring()

nx.draw(g, with_labels = True)

In [ ]:
nx.draw_networkx(
    g,
    pos = nx.drawing.layout.bipartite_layout(g, topnodes), 
    width = 1)
matching = bipartite.maximum_matching(g, top_nodes = topnodes)
mvc = bipartite.to_vertex_cover(g, matching, top_nodes=topnodes)
print(len(mvc))

In [ ]:
cg, topnodes = pacman.get_nx_graph_conflict()
nx.draw(cg, with_labels = True)

In [ ]:
nx.draw_networkx(
    cg,
    pos = nx.drawing.layout.bipartite_layout(cg, topnodes), 
    width = 1)